<img src="images/relampago_cacti_draft_logo-01.png" width=250 alt="Relampago_log"></img>
<!-- <img src="images/ARM_Logo.png" width=250 alt="Relampago_log"></img> -->

# Convection initiation in Sierras de Cordoba


---

## Overview

We will look at the 2019-01-29 convective storm near the Sierras de Cordoba

<video src="images/corlasso_animtb_2019012900_goes16band13.mp4"  width=500 controls></video>

## Science Question(s)
1) How is deep convection initiation controlled by local and regional meteorological conditions (Thunderstorm life cycle)as well as the geography?


## Project Scope (what does success look like)?
- Learning how to use LASSO simulatiom and ARM data. 
- Understand how convection is initiated in our domain by looking at measurements 
- Where convection is initiated in our domain of study
- Comparison between observation and summulations 

## Hypothesis (or Hypotheses)
- Convection initiation near the ARM site is caused largely by orographic effects in addition to surface heating and advection of moisture from the Pacific.

## Group Members
Eddie Wolff, Dhwanit J. Mise, Natalia Roldan, Alfonso Ladino, Victor Ojo

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Cartopy](https://foundations.projectpythia.org/core/cartopy/cartopy.html) | Necessary | |
| [Understanding of NetCDF](https://foundations.projectpythia.org/core/data-formats/netcdf-cf.html) | Helpful | Familiarity with metadata structure |
| Project management | Helpful | |

- **Time to learn**: estimate in minutes. For a rough idea, use 5 mins per subsection, 10 if longer; add these up for a total

---

## Imports
Let's import our required libraries

In [ ]:
import act
import xarray as xr
import cmweather
import matplotlib.pyplot as plt
import numpy as np
import pyart
import hvplot.xarray
import holoviews as hv
import glob
import xradar as xd
import cartopy.crs as ccrs
from dask.distributed import Client, LocalCluster
from functools import partial
hv.extension("bokeh")

## Dask Cluster
Let's spin up our local cluster using `Dask`

In [ ]:
client = Client(LocalCluster())

## Data acquisition

In [ ]:
username = "XXXXXX"  ## Use your ARM username
token = "XXXXX" ### Use your ARM token ()

We are going to analyze data from the **C-band Scanning ARM Precipitation Radar** [CSARP](https://www.arm.gov/capabilities/instruments/csapr) and the **Ka-band ARM Zenith Radar** [KZAR](https://www.arm.gov/capabilities/instruments/kazr). Data from both sensors can be found at the [ARM Data Discovery](https://adc.arm.gov/discovery/) portal under the porta data tab. Then, we can filter data by date, sensor, and field campaign. Once the desired data is located, we can get the identification. In our case, for the CSARP will be `corcsapr2cmacppiM1.c1` and for KZAR `corarsclkazr1kolliasM1.c1`

In [ ]:
# Set the datastream and start/enddates
kzar = 'corarsclkazr1kolliasM1.c1'
csarp = "corcsapr2cmacppiM1.c1"
startdate = '2019-01-29T13:00:00'
enddate = '2019-01-29T20:00:00'


Then, we can use the Atmospheric Data Community Toolkit [ACT](https://arm-doe.github.io/ACT/) to easily download the data. The [`act.discovery.download_arm_data`](https://arm-doe.github.io/ACT/API/generated/act.discovery.download_neon_data.html) module will allow us to download the data directly to our computer. Watch for the data citation! Show some support for ARM's instrument experts and cite their data if you use it in a publication.


In [ ]:
# CSARP files
# csapr_files = act.discovery.download_arm_data(username, token, csarp, startdate, enddate)

To examine the CACTI domain, we'll read in one timestep from the LASSO simulations and plot the terrain.

In [ ]:
path_staging = "/data/project/ARM_Summer_School_2024_Data/lasso_tutorial/cacti/lasso-cacti"  # path on Jupyter
file_list = sorted(glob.glob(f'{path_staging}/20190129/eda09/base/les/subset_d3/corlasso_met_*'))

ds = xr.open_dataset(file_list[50])
ds

We'll also plot the location of the CSAPR radar using the metadata from one of the radar files

In [ ]:
# Set the datastream and start/enddates
# Set your username and token here!
username = 'ecwolff'
token = '69d210b6616805b2'

datastream = 'corcsapr2cmacppiM1.c1'
startdate = '2019-01-29T18:00:00'
enddate = '2019-01-29T20:00:00'

# Use ACT to easily download the data.  Watch for the data citation!  Show some support
# for ARM's instrument experts and cite their data if you use it in a publication
result = act.discovery.download_arm_data(username, token, datastream, startdate, enddate)

In [ ]:
files = sorted(glob.glob('corcsapr2cmacppiM1.c1/*'))
radar = xd.io.open_cfradial1_datatree(files[2], first_dim="auto")
radar = radar.xradar.georeference()
display(radar)

In [ ]:
CSAPR_lat = float(radar.latitude.values)
CSAPR_lon = float(radar.longitude.values)

cf = plt.contourf(ds.XLONG, ds.XLAT, ds.HGT[0,:,:], cmap='terrain', levels=np.arange(0, 2500, 50))
plt.scatter(CSAPR_lon, CSAPR_lat, s=50, c='tab:red', label='ARM Site')
plt.title('CACTI Domain', fontsize=15)
plt.colorbar(cf, label='Height (m)')
plt.legend()

Now let's examine the convection occurring during this particular day. To do this, we'll use one of the radar files we imported above.

In [ ]:
import hvplot.xarray
import holoviews as hv
hv.extension('bokeh')

In [ ]:
vel_plot = radar['sweep_0']["mean_doppler_velocity"].hvplot.quadmesh(x="x", y="y", cmap="balance", 
                                                          height=300, width=400, 
                                                          clim=(-20, 20),
                                                          rasterize=True)

In [ ]:
ref_plot = radar['sweep_0']["reflectivity"].hvplot.quadmesh(x="x", y="y", cmap="ChaseSpectral", 
                                                          height=300, width=400, 
                                                          clim=(-20, 70),
                                                          rasterize=True)

In [ ]:
ref_plot+vel_plot

We can see that all of the storms seem to be forming along one particular area. It looks like this might be the mountain range in the center of the CACTI domain shown above, but let's overlay terrain on the radar image to verify this.

In [ ]:
proj_crs = xd.georeference.get_crs(radar["sweep_0"].ds)
cart_crs = cartopy.crs.Projection(proj_crs)

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection=cartopy.crs.PlateCarree())
radar["sweep_0"]["reflectivity"].plot(
    x="x",
    y="y",
    cmap="ChaseSpectral",
    transform=cart_crs,
    cbar_kwargs=dict(pad=0.075, shrink=0.75),
    vmin=-20,
    vmax=70
)
ax.coastlines()
ax.gridlines(draw_labels=True)

ax.set_xlim([-66,-63.5])
ax.set_ylim([-33.5, -31])

cf = plt.contour(ds.XLONG, ds.XLAT, ds.HGT[0,:,:], cmap='terrain', levels=np.arange(0, 2500, 350), 
                 transform=cartopy.crs.PlateCarree())

As we suspected, the storms are all located just east of the Sierras de Cordoba. This leads us to our hypothesis...

## Radar reflectivity plots


### CSARP

We found in total 15 files for the 2021-01-29 deep convective storms as follows

In [ ]:
files = sorted(glob.glob('corcsapr2cmacppiM1.c1/*'))
files[:3]

## Read the Data
We need to coarsen the azimuth angle, which willmake merging across different volume scans easier.

In [ ]:
def fix_angle(ds):
    """
    Aligns the radar volumes
    """
    ds["time"] = ds.time.load()  # Convert time from dask to numpy

    start_ang = 0  # Set consistent start/end values
    stop_ang = 360

    # Find the median angle resolution
    angle_res = ds.azimuth.diff("azimuth").median()

    # Determine whether the radar is spinning clockwise or counterclockwise
    median_diff = ds.azimuth.diff("time").median()
    ascending = median_diff > 0
    direction = 1 if ascending else -1

    # first find exact duplicates and remove
    ds = xd.util.remove_duplicate_rays(ds)

    # second reindex according to retrieved parameters
    ds = xd.util.reindex_angle(
        ds, start_ang, stop_ang, angle_res, direction, method="nearest"
    )

    ds = ds.expand_dims("volume_time")  # Expand for volumes for concatenation

    ds["volume_time"] = [np.nanmin(ds.time.values)]

    return ds

In [ ]:
dsets = []
for file in files:
    ds = xd.io.open_cfradial1_datatree(file)["sweep_0"].to_dataset()
    ds["azimuth"] = ds.azimuth.round(2)
    ds = fix_angle(ds)
    dsets.append(ds)

In [ ]:
ds = xr.concat(dsets, dim='volume_time')
ds

In [ ]:
ds = ds.xradar.georeference()

In [ ]:
proj_crs = xd.georeference.get_crs(ds)
cart_crs = ccrs.Projection(proj_crs)

In [ ]:
ds.reflectivity.hvplot.quadmesh(x='x',
                                          y='y',
                                          cmap='pyart_ChaseSpectral',
                                          groupby='volume_time',
                                          widget_type="scrubber",
                                          widget_location="bottom",
                                          clim=(-20, 70),
                                          width=600,
                                          height=500,
                                          rasterize=True)


In [ ]:
def rain_rate(ds, a=200, b=1.6, dt=5):
    z = ds.corrected_reflectivity
    z_lin = 10 ** (z / 10)
    ds['rr'] =((1 / a) ** (1 / b) * z_lin ** (1 / b)) * (dt/ 60) 
    return ds

In [ ]:
ds = rain_rate(ds, a=250, b=1.2, dt=15)
ds

### KZAR

In [ ]:
# KZAR files
# kzar_files = act.discovery.download_arm_data(username, token, kzar, startdate, enddate)

In [ ]:
# ds_kzar = act.io.read_arm_netcdf("corarsclkazr1kolliasM1.c1/corarsclkazr1kolliasM1.c1.20190129.000000.nc")
ds_kzar = xr.open_dataset("corarsclkazr1kolliasM1.c1/corarsclkazr1kolliasM1.c1.20190129.000000.nc")
display(ds_kzar)

As we are interested in the convective storm that occurred between the 16:00 and the 19:00 we can subset our dataset to only keep storm data

In [ ]:
ds_kzar = ds_kzar.sel(time=slice('2019-01-29 16:00', '2019-01-29 19:00'))

We would like see how radar reflectivity looks like in both instrument Therefore, we can use `hvplot` to create interactive plots 

In [ ]:
ref = ds_kzar.reflectivity_best_estimate.hvplot.quadmesh(x="time", y="height", 
                                                         cmap="ChaseSpectral", 
                                                         clim=(-20, 40),
                                                         rasterize=True, 
                                                         width=1000, 
                                                         height=300)
vel = ds_kzar.mean_doppler_velocity.hvplot.quadmesh(x="time", y="height", 
                                                         cmap="pyart_balance", 
                                                         clim=(-20, 20),
                                                         rasterize=True, 
                                                         width=1000, 
                                                         height=300)

In [ ]:
ref + vel


## CACTI Domain Overview

---

## Summary
Add one final `---` marking the end of your body of content, and then conclude with a brief single paragraph summarizing at a high level the key pieces that were learned and how they tied to your objectives. Look to reiterate what the most important takeaways were.

### What's next?
Let Jupyter book tie this to the next (sequential) piece of content that people could move on to down below and in the sidebar. However, if this page uniquely enables your reader to tackle other nonsequential concepts throughout this book, or even external content, link to it here!

## Resources and references
Finally, be rigorous in your citations and references as necessary. Give credit where credit is due. Also, feel free to link to relevant external material, further reading, documentation, etc. Then you're done! Give yourself a quick review, a high five, and send us a pull request. A few final notes:
 - `Kernel > Restart Kernel and Run All Cells...` to confirm that your notebook will cleanly run from start to finish
 - `Kernel > Restart Kernel and Clear All Outputs...` before committing your notebook, our machines will do the heavy lifting
 - Take credit! Provide author contact information if you'd like; if so, consider adding information here at the bottom of your notebook
 - Give credit! Attribute appropriate authorship for referenced code, information, images, etc.
 - Only include what you're legally allowed: **no copyright infringement or plagiarism**
 
Thank you for your contribution!